In [62]:
using Distributed
addprocs(4)

4-element Vector{Int64}:
 10
 11
 12
 13

In [63]:
@everywhere struct Parm
    #Parm of electrons
    t_i::Float64
    t_e::Float64
    t_aa::Float64
    t_bb::Float64
    Delta::Float64
    mu::Float64
    eta::Float64
    hx::Float64
    hy::Float64
    hz::Float64

    #Parm of phonon
    Cp::Float64
    m::Float64
    M::Float64
    U::Float64
    T::Float64
    K_SIZE::Int
    W_MAX::Float64
    W_SIZE::Int
    W_in::Float64
end

In [64]:
@everywhere using SharedArrays

In [65]:
@everywhere mutable struct Hamiltonian
    Hk::SharedArray{ComplexF64,3}
    Vx::SharedArray{ComplexF64,3}
    Vxx::SharedArray{ComplexF64,3}
    E::SharedArray{ComplexF64,2}
    #=
    Hk::Array{ComplexF64,2}
    Vx::Array{ComplexF64,2}
    Vxx::Array{ComplexF64,2}
    E::Array{ComplexF64,1}=#
end

@everywhere function HandV(p::Parm)
    Hk = SharedArray{ComplexF64,3}(p.K_SIZE,2,2)
    Vx = SharedArray{ComplexF64,3}(p.K_SIZE,2,2)
    Vxx = SharedArray{ComplexF64,3}(p.K_SIZE,2,2)
    E = SharedArray{ComplexF64,2}(p.K_SIZE,2)

    @distributed for k in 1:p.K_SIZE
        #k0::Float64 = 2.0*(k - p.K_SIZE/2)*pi/p.K_SIZE
        k0::Float64 = 2.0k*pi/p.K_SIZE
        eps::Float64 = -0.5*(p.t_aa + p.t_bb)*cos(k0)-p.mu
        g_x::Float64 = (p.t_e*cos(k0) -p.t_i*sin(k0)) - p.hx
        g_y::Float64 = (p.t_e*sin(k0) + p.t_i*cos(k0)) - p.hy
        g_z::Float64 = p.Delta - p.hz -0.5*(p.t_aa - p.t_bb)*cos(k0)
        gg = [eps, g_x, g_y, g_z]
        Hk[k,:,:] =  gg' * sigma

        eps_vx::Float64 = 0.5*(p.t_aa + p.t_bb)*sin(k0)
        gx_vx::Float64 = (-p.t_e*sin(k0) -p.t_i*cos(k0))
        gy_vx::Float64 = (p.t_e*cos(k0) - p.t_i*sin(k0))
        gz_vx::Float64 = 0.5*(p.t_aa - p.t_bb)*sin(k0)
        gg_x = [eps_vx, gx_vx, gy_vx, gz_vx]
        Vx[k,:,:] = gg_x' * sigma

        eps_vxx::Float64 = 0.5*(p.t_aa + p.t_bb)*cos(k0)
        gx_vxx::Float64 = (-p.t_e*cos(k0) +p.t_i*sin(k0))
        gy_vxx::Float64 = (-p.t_e*sin(k0) - p.t_i*cos(k0))
        gz_vxx::Float64 = 0.5*(p.t_aa - p.t_bb)*cos(k0)
        gg_xx = [eps_vxx, gx_vxx, gy_vxx, gz_vxx]
        Vxx[k,:,:] = gg_xx' * sigma

        E[k,:] = zeros(2)    
    end

    return Hk, Vx, Vxx, E 
end

In [66]:
@everywhere sigma = [[1.0 0.0; 0.0 1.0], [0.0 1.0; 1.0 0.0], [0.0 -1.0im; 1.0im 0.0], [1.0 0.0; 0.0 -1.0]]


In [67]:
p = Parm(1.0, 0.3, 0.1, 0.2, 0.1, -0.8, 0.02, 0.6, 0.0, 0.0, 0.5, 0.5, 1.0, 0.2, 0.02, 20, 1.5, 20, 1.0)

Parm(1.0, 0.3, 0.1, 0.2, 0.1, -0.8, 0.02, 0.6, 0.0, 0.0, 0.5, 0.5, 1.0, 0.2, 0.02, 20, 1.5, 20, 1.0)

In [83]:
H = Hamiltonian(HandV(p)...)

Hamiltonian([0.8048943483704847 + 0.0im -0.6237000394864014 + 1.0437616146076378im; 0.8190983005625052 + 0.0im -0.9450801539799889 + 0.9853525700626894im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;; -0.6237000394864014 - 1.0437616146076378im 0.5097886967409693 + 0.0im; -0.9450801539799889 - 0.9853525700626894im 0.5381966011250106 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.030901699437494747 + 0.0im -1.0437616146076378 - 0.02370003948640137im; 0.05877852522924733 + 0.0im -0.9853525700626894 - 0.3450801539799889im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;; -1.0437616146076378 + 0.02370003948640137im 0.061803398874989486 + 0.0im; -0.9853525700626894 + 0.3450801539799889im 0.11755705045849464 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.09510565162951537 + 0.0im 0.02370003948640137 - 1.0437616146076378im; 0.08090169943749476 + 0.0im 0.3450801539799889 - 0.9853525700626894im; … ; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0

In [84]:
size(H.Hk,1)

20

In [73]:
@everywhere using LinearAlgebra

In [86]:
H.E[3,:], BI = eigen(H.Hk[3,:,:])

Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 -0.7801326948388722
  2.20379711915113
vectors:
2×2 Matrix{ComplexF64}:
 -0.560425-0.377573im  0.611331+0.41187im
 -0.737131+0.0im       -0.67575+0.0im

In [87]:
H.Hk[3,:,:]

2×2 Matrix{ComplexF64}:
 0.841221+0.0im      -1.23268-0.83049im
 -1.23268+0.83049im  0.582443+0.0im

In [88]:
H.Hk[3,:,:] = [H.E[3,1] 0.0; 0.0 H.E[3,2]]

2×2 Matrix{ComplexF64}:
 -0.780133+0.0im     0.0+0.0im
       0.0+0.0im  2.2038+0.0im

In [89]:
Vx_BI = BI' * H.Vx[3,:,:] * BI

2×2 Matrix{ComplexF64}:
 -0.209125+0.0im       0.0693725+0.988741im
 0.0693725-0.988741im    0.45183+1.11022e-16im

In [90]:
@distributed for k in 1:size(H.Hk,1)
    H.E[k,:], BI = eigen(H.Hk[k,:,:])
    H.Hk[k,:,:] = [H.E[k,1] 0.0; 0.0 H.E[k,2]]
    Vx_BI = BI' * H.Vx[k,:,:] * BI
    Vxx_BI = BI' * H.Vxx[k,:,:] * BI


    H.Vx[k,:,:] = Vx_BI
    H.Vxx[k,:,:] = Vxx_BI
end

Task (runnable) @0x000000010bf03820

In [20]:
@everywhere mutable struct Green
    GR::SharedArray{ComplexF64,4}
    GA::SharedArray{ComplexF64,4}
    GRmA::SharedArray{ComplexF64,4}
    #=
    GR::Array{ComplexF64,2}
    GA::Array{ComplexF64,2}
    GRmA::Array{ComplexF64,2}
    GRp::Array{ComplexF64,2}
    GAp::Array{ComplexF64,2}
    GRm::Array{ComplexF64,2}
    GAm::Array{ComplexF64,2}=#
end


Gk (generic function with 1 method)

In [21]:

@everywhere using LinearAlgebra

In [23]:

@everywhere function Gk(Ham::Hamiltonian, p::Parm)

    GR = SharedArray{ComplexF64,4}(p.K_SIZE,p.W_SIZE,2,2)
    GA = SharedArray{ComplexF64,4}(p.K_SIZE,p.W_SIZE,2,2)
    GRmA = SharedArray{ComplexF64,4}(p.K_SIZE,p.W_SIZE,2,2)

    @distributed for k in 1:p.K_SIZE
        for w in 1:p.W_SIZE
            w0 = 2.0*(w - p.W_SIZE/2)*p.W_MAX/p.W_SIZE
            GR0::Array{ComplexF64,2} = -Ham.Hk[k,:,:] + Matrix{Complex{Float64}}(w0*I,2,2) + p.eta*Matrix{Complex{Float64}}(1.0im*I,2,2)
            GR[k,w,:,:] = inv(GR0)
            GA[k,w,:,:] = GR[k,w,:,:]'
            GRmA[k,w,:,:] = GR[k,w,:,:] - GA[k,w,:,:]
        end
    end
    return GR, GA, GRmA
end

In [24]:
G = Green(Gk(H,p)...)

Green([0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im;;;; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.

In [47]:
G.GA[1,10,:,:]

2×2 Matrix{ComplexF64}:
 0.476352+0.0304395im  0.559332+0.990599im
 0.607378-0.961889im   0.752368+0.0372316im

In [59]:
function PV_calcu_simple(p::Parm, H::Hamiltonian, G::Green, Jxx::SharedArray{Float64,1})
    #@everywhere dk::Float64 = 2pi/p.K_SIZE
    #@everywhere dw::Float64 = 2*p.W_MAX/p.W_SIZE
    #@everywhere wi::Int = 10
    #convert(Int,round(p.W_in/dw))

    #Jxxx = SharedArray{Float64,1}(p.K_SIZE)

    
    @distributed for k in 1:p.K_SIZE
        dk::Float64 = 2pi/p.K_SIZE
        dw::Float64 = 2*p.W_MAX/p.W_SIZE
        wi::Int = 10
        for w in 1:p.W_SIZE
            ww = 2.0p.W_MAX*(w-p.W_SIZE/2)/p.W_SIZE
            Jxxx[k] += dk * dw * imag(tr(H.Vx[k,:,:] * G.GR[k,w,:,:] * H.Vx[k,:,:] * (G.GR[k,w+wi,:,:] + G.GR[k,w-wi,:,:]) * H.Vx[k,:,:] * G.GRmA[k,w,:,:])) * f(ww,p.T)
            Jxxx[k] += dk * dw * imag(tr(H.Vx[k,:,:] * G.GR[k,w,:,:] * H.Vxx[k,:,:] * G.GRmA[k,w,:,:])) * f(ww,p.T)
            Jxxx[k] += dk * dw * imag(tr(H.Vx[k,:,:] * G.GR[k,w-wi,:,:] * H.Vx[k,:,:] * (G.GRmA[k,w,:,:]) * H.Vx[k,:,:] * G.GA[k,w-wi,:,:])) * f(ww,p.T)
            Jxxx[k] += dk * dw * imag(tr(H.Vx[k,:,:] * G.GR[k,w+wi,:,:] * H.Vx[k,:,:] * (G.GRmA[k,w,:,:]) * H.Vx[k,:,:] * G.GA[k,w+wi,:,:])) * f(ww,p.T)
            Jxxx[k] += dk * dw * imag(tr(H.Vx[k,:,:] * G.GRmA[k,w,:,:] * H.Vx[k,:,:] * (G.GR[k,w+wi,:,:] + G.GR[k,w-wi,:,:]) * H.Vx[k,:,:] * G.GA[k,w,:,:])) * f(ww,p.T)
            Jxxx[k] += dk * dw * imag(tr(H.Vx[k,:,:] * G.GRmA[k,w,:,:] * H.Vxx[k,:,:] * G.GA[k,w,:,:])) * f(ww,p.T)
        end
        if(k==1)
            println("check!")
            println(Jxxx[k])
        end
    end
    #println("Hello")
    #return Jxxx/(2*pi)/(p.W_in^2)
end

PV_calcu_simple (generic function with 2 methods)

In [33]:
PV_XXX_mu = SharedArray{Float64,1}(p.K_SIZE)
#PV_calcu_simple(p, H, G, PV_XXX_mu)

20-element SharedVector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [60]:
 #test = PV_calcu_simple(p, H, G)
 #, PV_XXX_mu)
PV_calcu_simple(p, H, G, PV_XXX_mu)

Task (runnable) @0x000000010d1ebde0

In [61]:
PV_XXX_mu[5]

Unhandled Task ERROR: On worker 2:
UndefVarError: Jxxx not defined
Stacktrace:
 [1] macro expansion
   @ ~/Documents/Codes/julia/test1.ipynb:16 [inlined]
 [2] #35
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Distributed/src/macros.jl:303
 [3] #172
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Distributed/src/macros.jl:83
 [4] #103
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Distributed/src/process_messages.jl:274
 [5] run_work_thunk
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Distributed/src/process_messages.jl:63
 [6] run_work_thunk
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Distributed/src/process_messages.jl:72
 [7] #96
   @ ./task.jl:423

...and 7 more exceptions.

Stacktrace:
 [1] sync_end(c::Channel{Any})
   @ Base ./task.jl:381
 [2] (::Distributed.var"#171#173"{var"#35#36"{Parm, Hamiltonian, Gr

0.0